In [53]:
import os 
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores import Neo4jVector
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pydantic import BaseModel, Field
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import  RunnablePassthrough
from typing import List
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface import HuggingFaceEmbeddings
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
from huggingface_hub import login
from dotenv import load_dotenv
import json
import warnings
load_dotenv()

# login()

model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

# llm = ChatOpenAI(temperature=0.8, model="gpt-3.5-turbo")

In [7]:
os.environ['NEO4J_URI'] = 'neo4j+s://0a95ce3e.databases.neo4j.io'
os.environ['NEO4J_USERNAME'] = 'neo4j'
os.environ['NEO4J_PASSWORD'] = '1SgL9cPF7fKii-Tmrm231WbxAgxZ3Q-0mfnsHMriRoI'
os.environ['NEO4J_DATABASE'] = 'neo4j'
os.environ['AURA_INSTANCEID'] = '0a95ce3e'
os.environ['AURA_INSTANCENAME'] = 'Instance01'

graph = Neo4jGraph()

In [3]:
# from pypdf import PdfMerger

# pdfs = ['wsu_courses.pdf','international_guide.pdf','international.pdf','english_requirements.pdf']

# merger = PdfMerger()

# for pdf in pdfs:
#     merger.append(pdf)

# merger.write("wsu-data.pdf")
# merger.close()

In [4]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("wsu-data.pdf")
# documents = loader.load()

In [5]:
# # a simple function that removes \n newline from the content
# def remove_ws(d):
#     text = d.page_content.replace('\n','')
#     d.page_content = text
#     return d

# # applied on the docs
# documents = [remove_ws(d) for d in documents]

In [6]:
# from langchain_experimental.graph_transformers import LLMGraphTransformer

# llm_transformer = LLMGraphTransformer(llm=llm)
# graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [7]:
# import pickle

# # Save graph_documents as a Pickle file
# with open("graph_enhanced.pkl", "wb") as file:
#     pickle.dump(graph_documents, file)

In [9]:
import pickle

# Open the saved graph documents
with open('graph_enhanced.pkl', 'rb') as file:
    graph_documents = pickle.load(file)

# Open the saved graph documents
with open('ultimate_graph.pkl', 'rb') as file:
    graph_documents2 = pickle.load(file)

In [9]:
# graph.add_graph_documents(
#     graph_documents,
#     baseEntityLabel=True,
#     include_source=True
# )

# graph.add_graph_documents(
#     graph_documents2,
#     baseEntityLabel=True,
#     include_source=True
# )

In [10]:
# directly show the graph resulting from the given Cypher query:
# default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

# def showGraph(cypher: str = default_cypher):
#     driver = GraphDatabase.driver(
#         uri = os.environ["NEO4J_URI"],
#         auth = (os.environ["NEO4J_USERNAME"],
#                 os.environ["NEO4J_PASSWORD"]))
#     session = driver.session()
#     widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
#     widget.node_label_mapping = 'id'
#     return widget

# showGraph()

In [11]:
# vector_index = Neo4jVector.from_existing_graph(
#     OpenAIEmbeddings(model = "text-embedding-3-large"),
#     #OllamaEmbeddings(model = "mxbai-embed-large"),
#     search_type="hybrid",
#     node_label="Document",
#     text_node_properties=["text"],
#     embedding_node_property="embedding"
# )
# vector_retriever = vector_index.as_retriever()

index_name = "vector"
keyword_index_name = "keyword"

vector_index = Neo4jVector.from_existing_index(
    model,
    index_name = index_name,
    keyword_index_name=keyword_index_name,
    search_type="hybrid"
)

retriever = vector_index.as_retriever()

In [12]:
vector_index.similarity_search(query = "Liverpool City campus")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN node.`text` AS text, score, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata'


[Document(metadata={'source': 'wsu-data.pdf', 'page': 29}, page_content='SAFEGUARD SOCIETYDegrees in Criminology, Policing and Cyber Security can set you up for a great career in the police force, criminal and community justice systems, corrective services, border protection, intelligence, cyber security, data analytics or forensics or computer forensics. A career in these fields can be both challenging and rewarding. The future of cyber security relies on people like you who are able to think quickly, find innovative solutions, and work to keep the data of companies and nations secure, as well as protect our individual privacy. It is a rapidly growing field that is essential for the smooth operation of businesses, organisations and governments in today’s connected world. Western Sydney University’s Policing degree is the only one based in metropolitan Sydney. We have strong ties with police recruiters increasing your chances of joining the NSW, Victorian or Australian Federal Police. 

In [14]:
graph.query("""
  SHOW VECTOR INDEXES
  """
)

[{'id': 2,
  'name': 'vector',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Document'],
  'properties': ['embedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 10, 28, 1, 37, 12, 877000000, tzinfo=<UTC>),
  'readCount': 1}]

In [15]:
graph.refresh_schema()

In [16]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)
# Entities_converted = convert_to_openai_function(Entities)
structured_llm = llm.with_structured_output(Entities)
entity_chain = prompt | structured_llm

# # Function to ensure entities.names is a list
# def ensure_list(entities_names):
#     if isinstance(entities_names, str):
#         return json.loads(entities_names)
#     return entities_names

In [17]:
entity_chain.invoke({"question": "Western Sydney"}).names

['Western Sydney']

In [18]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.
    
    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and combines them
    using the AND operator, ensuring exact matches without allowing misspellings.
    """
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join(words)
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    if not any(index["name"] == "entity" for index in graph.query("SHOW INDEXES")):
        graph.query("CREATE FULLTEXT INDEX entity FOR (n:__Entity__) ON EACH [n.id]")
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """
            CALL db.index.fulltext.queryNodes('entity', $query)
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [20]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document". join(vector_data)}
    """
    return final_data

In [64]:
from langchain_core.runnables import  RunnablePassthrough

parser = StrOutputParser()

template = """Answer the question based only on the following context:
{context}

Question: {question}
- You are a representative at Western Sydney University, your job is to provide accurate information, and provide accurate information about Western Sydney University to \
international students who wish to enrol at the university.
- Use natural language.
- Extend your answers, try to connect to neighbor entities for more contextually aware response.
- If there's no entity matching the query, return "There is no information in the database regarding what \
you are requesting. Perhaps you should check your spelling or try a different prompt."

Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | parser
)

In [29]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [65]:
# print(graph_retriever("Dibya Shree Chhetry"))
 
query = str(input("Enter your query:"))
# print(graph_retriever(query))
answer = (chain.invoke(input = query))
retrieved_doc = compression_retriever.invoke(query)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 4, column: 13, offset: 106} for query: "\n            CALL db.index.fulltext.queryNodes('entity', $query)\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "


Generated Query: bachelor AND of AND applied AND data AND science
Generated Query: subjects


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 4, column: 13, offset: 106} for query: "\n            CALL db.index.fulltext.queryNodes('entity', $query)\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientN

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 18088 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [61]:
def full_answer(a,b):
    return print(answer), retrieved_doc

full_answer(answer, retrieved_doc)

The subjects for the Bachelor of Applied Data Science program at Western Sydney University include Thinking About Data, Analytics Programming, Visual Analytics, Applications of Big Data, Social Web Analytics, Discovery Project, and Predictive Modelling. These subjects provide students with a strong foundation in mathematics, statistics, and computing, allowing them to embark on data-driven investigations and conduct visual and computational analytics. The program is designed to be undertaken in combination with any bachelor degree, providing students with the opportunity to enhance their skills in data science while pursuing their primary discipline. The Bachelor of Applied Data Science is a unique program that equips graduates with the knowledge and skills required to excel in a data-driven world, opening up a wide range of career opportunities in commercial and government institutions.


(None,
 [Document(metadata={'source': 'wsu-data.pdf', 'page': 101}, page_content='Bachelor of Data Science is one of a kind. It is unique because it can only be studied in conjunction with another degree. Blend Data Science with courses like Marketing, Engineering, Information and Communications Technology, Science, or Psychology to graduate with a combined degree.')])